# Pandas and duckdb

## Task 0 - Load the sakila sqlite database into a duckdb database file

In [ ]:
import duckdb 

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC").df()

description

# Read all data into dictionary of pandas dataframes

In [ ]:
dfs = {}

with duckdb.connect("data/sakila.duckdb") as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM staging.{name};").df()
        duckdb.register(name, dfs[name])

dfs.keys()

## Task 1

### Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [ ]:
long_movies = duckdb.sql("""
SELECT title, length AS duration
FROM films
WHERE length > 180; 
""").df()

long_movies.head()

### Which movies have the word "love" in its title? Show the following columns
#### title
#### rating
#### length
#### description

In [ ]:
love_word = duckdb.sql("""
SELECT title, rating, length, description 
FROM films
WHERE title ILIKE '%love%'; 
""").df()

love_word.head()

### Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

In [ ]:
aggregate_function = duckdb.sql("""
SELECT 
    MIN(length) AS min_length_movies, 
    ROUND(AVG(length)) AS mean_length_movies, 
    MEDIAN(length) AS median_length_movies, 
    MAX(length) AS max_length_movies
FROM films
""").df()


aggregate_function

### The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie. The Manager wants to know the 10 most expensive movies to rent per day.

In [ ]:
rental_rate = duckdb.sql("""
SELECT title, rental_rate, rental_duration, (rental_rate / rental_duration) AS cost_per_day                
FROM films
ORDER BY cost_per_day DESC;
""").df()

rental_rate.head(10)

In [ ]:
description

###  Which actors have played in most movies? Show the top 10 actors with the number of movies they have played in.

In [ ]:
top_10_actors = duckdb.sql("""
SELECT a.actor_id, a.first_name, a.last_name, COUNT(*) AS film_count                
FROM films_joined 
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY fil_count DESC;
""").df()

top_10_actors